# Data Exploration

In [ ]:
#yang pertama kita lakukan adalah membaca file csv 
#lalu kita tampilkan 5 record teratas menggunakaan fungsi head

In [2]:
import pandas as pd
df = pd.read_csv('order_brush_order.csv')
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [ ]:
#setelah itu kita cari berapa jumlah shopid secara keseluruhan , ternyata jumlah kolom shopid adalah 18870

In [3]:
df.groupby('shopid').size().describe()

count    18770.000000
mean        11.867342
std        104.244723
min          1.000000
25%          1.000000
50%          2.000000
75%          8.000000
max      11703.000000
dtype: float64

In [ ]:
#selanjutnya kita liat shop (toko) mana yang ordernya paling tinggi 

In [4]:
df.groupby('shopid')['orderid'].count().sort_values(ascending=False).head(10)

shopid
147941492    11703
61556313      6691
54615708      2640
91799978       707
57189823       687
107921853      684
25924280       655
89768730       607
43412276       529
32504026       516
Name: orderid, dtype: int64

# Data Analysis

In [ ]:
#Temukan semua shops dan shops id yang memiliki lebih dari 3 order

In [8]:
unique_shopid = df['shopid'].unique()
print(f'{len(unique_shopid )} shops unique')

shopid_lebihdari3_df = pd.DataFrame(df.groupby('shopid').size()[df.groupby('shopid').size() >= 3]).to_dict()
shopid_lebihdari3_df = shopid_lebihdari3_df[0].keys()
print(f'{len(shopid_lebihdari3_df)} shop lebih dari 3 orders')


18770 shops unique
9031 shop lebih dari 3 orders


In [ ]:
#membuat timestamp

In [9]:
df['event_time'] = pd.to_datetime(df['event_time'])
df['ts'] = df[['event_time']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)
df.head()

,orderid,shopid,userid,event_time,ts
0,31076582227611,93950878,30530270,2019-12-27 00:23:03,1577406183
1,31118059853484,156423439,46057927,2019-12-27 11:54:20,1577447660
2,31123355095755,173699291,67341739,2019-12-27 13:22:35,1577452955
3,31122059872723,63674025,149380322,2019-12-27 13:01:00,1577451660
4,31117075665123,127249066,149493217,2019-12-27 11:37:55,1577446675


In [ ]:
#urutkan orders berdasarkan event time

In [10]:
df = df.sort_values(by='ts', ascending=True)
df

,orderid,shopid,userid,event_time,ts
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,1577404800
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,1577404800
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,1577404800
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,1577404802
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,1577404805
...,...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43,1577836783
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47,1577836787
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51,1577836791
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51,1577836791


In [ ]:
#Mulai untuk mendeteksi brushing orders 

Variable definition: 
    1. previous_order : Order terdekat sebelum time interval.
    2. previous_order_time : Event time dari previous_order.
    3. start_order : Order pertama interval waktu ini.
    4. start_order_time : Event time dari start_order.
    5. end_order :  Order terakhir interval waktu ini.
    6. end_order_time: Event time dari end_order.
    7. next_order : Order terdekat setelah time interval.
    8. next_order_time: Event time dari next_order.
        
Kita akan menemukan time interval dengan 2 kondisi berikut :
    1. start_order_time dan end_order_time dalam waktu 1 jam.
    2. next_order_time - previous_order_time > 1 jam.


In [15]:
from datetime import datetime
import time 

count = 0
ans_dict = dict()

#lakukan pengulangan berdasarkan banyaknya shopid yang memiliki order > 3
for shop_id in shopid_lebihdari3_df : 

    time_record = time.time()
    cheat = 0
    
    #record the cheating row index in shop_df
    cheat_order_list = set()
    
    shop_df = df[df['shopid'] == shop_id]
    len_shop_data = len(shop_df)
    
    #tetapkan variabel previous_order_time yang merupakan 1 jam lebih awal dari start_order_time,di sini kami menetapkan 9999 detik yang lebih besar dari 3600 detik (alias 1 jam)
    previous_order_time = shop_df.iloc[0]['ts'] - 9999
    
    #Kami tidak perlu memeriksa 2 pesanan terakhir karena perlu lebih dari 3 pesanan agar sesuai dengan kondisi order brushing
    
    for start_order in range(len_shop_data - 2): 
        start_order_time = shop_df.iloc[start_order]['ts']
        
        # Sama seperti alasan diatas, end_order dimulai dari start_order + 2
        for end_order in range(start_order + 2, len_shop_data): 
            
            # start_order_time dan end_order_time perlu dalam 1 jam
            end_order_time = shop_df.iloc[end_order]['ts']
            if end_order_time > start_order_time + 3600:
                break
                
            # Temukan event time dari next_order
            # Kondisi edge dimana end_order adalah order terakhir dari toko ini
            
            if end_order == len_shop_data -1:
                # mirip dengan yang kami lakukan untuk previous_order_time
                next_order_time = shop_df.iloc[end_order]['ts'] + 9999 
            else:
                 next_order_time = shop_df.iloc[end_order + 1]['ts']
                    
                     # check whether next_order_time - previous_order_time > 1 hour
            if next_order_time > previous_order_time + 3601:
                # we can not divide orders which have same timestamp into different time interval
                if next_order_time == end_order_time: 
                    continue

                tmp_df = shop_df.iloc[start_order: end_order + 1]
                order_num = tmp_df.orderid.nunique()
                user_num = tmp_df.userid.nunique()

                # check whether order brushing
                if order_num / user_num >= 3:
                    cheat = 1
                    cheat_order_list.update(list(range(start_order, end_order + 1)))
        # refresh the previous order time
        previous_order_time = start_order_time 
        
    count += 1 

    if cheat == 1:
        # find the brushing user
        tmp_df = shop_df.iloc[list(cheat_order_list)]
        tmp_dict = tmp_df.groupby('userid').size().to_dict()
        max_time = max(list(tmp_dict.values()))
        
        ans_dict[shop_id] = []
        for user in tmp_dict:
            if tmp_dict[user] == max_time:
                ans_dict[shop_id].append(user)
                        
        print(f'{count:4d}, shop {shop_id} used {time.time() - time_record:4.2f} sec(s), cheat!!!')

   8, shop 10159 used 0.05 sec(s), cheat!!!
  27, shop 10402 used 0.00 sec(s), cheat!!!
  37, shop 10536 used 0.01 sec(s), cheat!!!
  67, shop 42472 used 0.01 sec(s), cheat!!!
  69, shop 42818 used 0.04 sec(s), cheat!!!
  77, shop 76934 used 0.00 sec(s), cheat!!!
 101, shop 195531 used 0.01 sec(s), cheat!!!
 136, shop 425364 used 0.00 sec(s), cheat!!!
 148, shop 599533 used 0.01 sec(s), cheat!!!
 149, shop 605561 used 0.01 sec(s), cheat!!!
 167, shop 722132 used 0.03 sec(s), cheat!!!
 169, shop 731606 used 0.00 sec(s), cheat!!!
 170, shop 736620 used 0.01 sec(s), cheat!!!
 178, shop 769445 used 0.05 sec(s), cheat!!!
 186, shop 823357 used 0.01 sec(s), cheat!!!
 226, shop 1175477 used 0.06 sec(s), cheat!!!
 241, shop 1532569 used 0.00 sec(s), cheat!!!
 292, shop 2831130 used 0.02 sec(s), cheat!!!
 293, shop 2856708 used 0.03 sec(s), cheat!!!
 309, shop 3124091 used 0.00 sec(s), cheat!!!
 332, shop 3701311 used 0.01 sec(s), cheat!!!
 348, shop 4149345 used 0.06 sec(s), cheat!!!
 377, sho

4552, shop 114016125 used 0.00 sec(s), cheat!!!
4579, shop 114919380 used 0.23 sec(s), cheat!!!
4593, shop 115179739 used 0.12 sec(s), cheat!!!
4600, shop 115506347 used 0.03 sec(s), cheat!!!
4659, shop 116337057 used 0.01 sec(s), cheat!!!
4672, shop 116348362 used 0.02 sec(s), cheat!!!
4721, shop 117079735 used 0.01 sec(s), cheat!!!
4749, shop 118139770 used 0.01 sec(s), cheat!!!
4784, shop 118949192 used 0.01 sec(s), cheat!!!
4812, shop 119309887 used 0.01 sec(s), cheat!!!
4837, shop 119362482 used 0.04 sec(s), cheat!!!
4863, shop 119595148 used 0.01 sec(s), cheat!!!
4873, shop 119690532 used 0.01 sec(s), cheat!!!
4886, shop 120358496 used 0.05 sec(s), cheat!!!
4895, shop 120692254 used 0.01 sec(s), cheat!!!
4972, shop 122238789 used 0.00 sec(s), cheat!!!
4982, shop 122459518 used 0.02 sec(s), cheat!!!
5003, shop 123401085 used 0.00 sec(s), cheat!!!
5007, shop 123548863 used 0.01 sec(s), cheat!!!
5047, shop 126261266 used 0.01 sec(s), cheat!!!
5059, shop 126587078 used 0.01 sec(s), c

In [16]:
# count how many shop is brushing (tips: the true num is 315 shops)
len(ans_dict)

315

In [17]:
ans_shop = []
ans_user = []
for shop in ans_dict.keys():
    if len(ans_dict[shop]) > 1:
        user_cheat = set(ans_dict[shop])
        user_cheat = sorted(list(user_cheat), reverse=False)
        ans_user.append("&".join([str(i) for i in user_cheat]))
    else:
        ans_user.append(str(ans_dict[shop][0]))
    ans_shop.append(shop)

for shop in unique_shopid:
    if shop not in ans_dict.keys():
        ans_shop.append(shop)
        ans_user.append("0")

print(len(ans_shop))
print(len(ans_user))

df_ans = pd.DataFrame({'shopid': ans_shop, 'userid': ans_user})
df_ans.to_csv('prediction.csv',index=False)

18770
18770
